# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119423e+02     1.611731e+00
 * time: 0.5074131488800049
     1     1.146934e+01     1.012074e+00
 * time: 1.1238250732421875
     2    -1.195142e+01     1.075805e+00
 * time: 1.1834700107574463
     3    -3.428890e+01     8.818910e-01
 * time: 1.2678911685943604
     4    -4.799133e+01     6.366852e-01
 * time: 1.3527541160583496
     5    -5.696223e+01     1.886577e-01
 * time: 1.5274081230163574
     6    -5.986099e+01     1.189047e-01
 * time: 1.5786340236663818
     7    -6.090171e+01     6.870723e-02
 * time: 1.6306710243225098
     8    -6.131526e+01     7.746972e-02
 * time: 1.6823642253875732
     9    -6.162907e+01     3.717340e-02
 * time: 1.7340970039367676
    10    -6.185329e+01     3.055578e-02
 * time: 1.7865240573883057
    11    -6.201562e+01     2.382769e-02
 * time: 2.214853048324585
    12    -6.208052e+01     1.992308e-02
 * time: 2.266803026199341
    13    -6.213166e+01     1.674111e-02
 * time: 2.318235158920

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671084
    AtomicLocal         -18.8557715
    AtomicNonlocal      14.8522655
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485391 
    Xc                  -19.3336828

    total               -62.261666460870